#Introdução

Este notebook apresenta a modelagem dimensional (Esquema Estrela) dos dados de emendas parlamentares, com o objetivo de estruturar a camada Gold para análises analíticas e financeiras.

A partir das tabelas da camada Silver — emendas e parlamentares — são construídas uma tabela fato e suas tabelas de dimensão, organizando as informações em torno das métricas financeiras (valores empenhados, liquidados e pagos) e dos principais eixos analíticos: quem, quando, onde e em quê os recursos foram destinados.

Essa modelagem visa otimizar consultas, facilitar análises históricas e comparativas e garantir uma base consistente para consumo por ferramentas analíticas e relatórios.

**Para maior detalhamento da modelagem dimensional, acessar o markdown na pasta de documentações** 

# Configuração do notebook

## Importar bibliotecas

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

## Carregar bases das silver

In [0]:
emendas_silver = spark.read.table("mvp.silver.emendas")
parlamentares_silver = spark.read.table("mvp.silver.parlamentares")

# Criação das tabelas de dimensão

## Dim_Parlamentar (Quem)

### 👤 Dimensão: Dim_Parlamentar

| Coluna           | Descrição                                                      | Tipo de Dado | Observações |
|------------------|----------------------------------------------------------------|--------------|-------------|
| sk_parlamentar   | Chave substituta (surrogate key) da dimensão parlamentar.      | BIGINT       | Gerada na camada Gold |
| nome_parlamentar | Nome do autor da emenda.           | STRING       | Chave natural para deduplicação |
| sigla_partido    | Sigla do partido político do parlamentar.                      | STRING       | Valor padronizado |
| sigla_uf         | Unidade da Federação representada pelo parlamentar.            | STRING       | Sigla da UF |


In [0]:
# Criação da coluna de surrogate
parlamentares_silver = (
    parlamentares_silver
    .withColumn("sk_parlamentar", F.monotonically_increasing_id())
)

In [0]:
# Reorganizar as colunas
parlamentares_silver = parlamentares_silver.select(
    "sk_parlamentar",
    "nome_parlamentar",
    F.col("siglaPartido").alias("sigla_partido"),
    F.col("siglaUf").alias("sigla_uf")
)

In [0]:
# Salvar dimensão na camada gold
(
    parlamentares_silver
    .write
    .mode("overwrite")
    .format("delta")
    .saveAsTable("mvp.gold.dim_parlamentar")
)

In [0]:
spark.sql("""
COMMENT ON TABLE mvp.gold.dim_parlamentar IS
'Dimensão parlamentar. Consolida informações de identificação e contexto político dos autores das emendas parlamentares.'
""")

DataFrame[]

In [0]:
spark.sql("""
COMMENT ON COLUMN mvp.gold.dim_parlamentar.sk_parlamentar IS
'Chave substituta (surrogate key) da dimensão parlamentar, gerada na camada Gold.'
""")

spark.sql("""
COMMENT ON COLUMN mvp.gold.dim_parlamentar.nome_parlamentar IS
'Nome do parlamentar autor da emenda padronizado. Utilizado como chave natural para deduplicação.'
""")

spark.sql("""
COMMENT ON COLUMN mvp.gold.dim_parlamentar.sigla_partido IS
'Sigla do partido político do parlamentar.'
""")

spark.sql("""
COMMENT ON COLUMN mvp.gold.dim_parlamentar.sigla_uf IS
'Unidade da Federação representada pelo parlamentar (sigla da UF).'
""")

DataFrame[]

In [0]:
# Verificar duplicidade de chave
parlamentares_silver.groupBy("sk_parlamentar").count().filter("count > 1").display()

sk_parlamentar,count


In [0]:
# Verificar cardinalidade
parlamentares_silver.count()

512

In [0]:
# Avaliando os parlamentares da dimensão parlamentar que não tem nenhuma emenda associada
df_parlamentares_sem_emendas = (
    parlamentares_silver.alias("dim")
    .join(
        emendas_silver.alias("emd"),
        on="nome_parlamentar",
        how="left_anti"
    )
)

In [0]:
df_parlamentares_sem_emendas.display()

nome_parlamentar,sk_parlamentar,sigla_partido,sigla_uf
ALLAN GARCES,23,PP,MA
CAIO VIANNA,66,PSD,RJ
DOUGLAS VIEGAS,144,UNIÃO,SP
ENFERMEIRA ANA PAULA,168,PODE,CE
ENFERMEIRA REJANE,169,PCdoB,RJ
FABIANO CAZECA,177,PRD,MG
FATIMA PELAES,182,REPUBLICANOS,AP
FAUSTO JR.,183,UNIÃO,AM
GISELA SIMONA,217,UNIÃO,MT
GUILHERME DERRITE,220,PP,SP


In [0]:
%sql
SELECT *
FROM mvp.silver.emendas
WHERE nome_parlamentar like "%REJANE%"

nome_parlamentar,ano,funcao,localidade_destino,municipio_destino,uf_destino,valor_empenhado,valor_liquidado,valor_pago
REJANE DIAS,2023,Educação,PIAUÍ (UF),null,PI,814604.0,0.0,0.0
REJANE DIAS,2023,Saúde,PIAUÍ (UF),null,PI,7504032.0,7504032.0,7504032.0
REJANE DIAS,2023,Saúde,CASTELO DO PIAUÍ - PI,CASTELO DO PIAUÍ,PI,2460396.0,299998.0,299998.0
REJANE DIAS,2023,Saúde,TERESINA - PI,TERESINA,PI,2443813.0,2443813.0,2443813.0
REJANE DIAS,2023,Encargos especiais,MÚLTIPLO,null,null,1.5237094E7,1.5237094E7,1.5237094E7
REJANE DIAS,2023,Saúde,MÚLTIPLO,null,null,2030602.0,2030602.0,2030602.0


A Enfermeira Rejane e a Rejane Dias são pessoas diferentes

In [0]:
%sql
SELECT *
FROM mvp.silver.emendas
WHERE nome_parlamentar like "%DERRITE%"

nome_parlamentar,ano,funcao,localidade_destino,municipio_destino,uf_destino,valor_empenhado,valor_liquidado,valor_pago
CAPITAO DERRITE,2023,Desporto e lazer,SÃO PAULO (UF),null,SP,400000.0,399994.3,399994.3
CAPITAO DERRITE,2023,Assistência social,MÚLTIPLO,null,null,3645000.0,3645000.0,3645000.0
CAPITAO DERRITE,2023,Defesa nacional,NACIONAL,null,null,662920.0,500000.0,500000.0
CAPITAO DERRITE,2023,Assistência social,SÃO PAULO (UF),null,SP,162920.0,162920.0,162920.0
CAPITAO DERRITE,2023,Saúde,MÚLTIPLO,null,null,1.3784169E7,1.3784169E7,1.3784169E7
CAPITAO DERRITE,2023,Saúde,SÃO PAULO (UF),null,SP,420000.0,0.0,0.0
CAPITAO DERRITE,2023,Encargos especiais,MÚLTIPLO,null,null,1.1178387E7,1.1178387E7,1.1178387E7
CAPITAO DERRITE,2023,Saúde,ALUMÍNIO - SP,ALUMÍNIO,SP,244566.0,244566.0,244566.0


O Guilherme Derrite está com divergência entre as nomenclaturas e esses casos não conseguiram ser avaliados em nossas análises.

Ao todo são 29 deputados federais que não vamos conseguir identificar nenhuma emendas em nossa tabela fato.

## Dim_Tempo (Quando)

### 📅 Dimensão: Dim_Tempo

| Coluna    | Descrição                                           | Tipo de Dado | Observações |
|-----------|------------------------------------------------------|--------------|-------------|
| sk_tempo  | Chave substituta (surrogate key) da dimensão tempo.  | BIGINT       | Gerada na camada Gold |
| ano       | Ano de referência da emenda parlamentar.             | INT          | Granularidade anual |


In [0]:
# Construção da Dim_Tempo
df_dim_tempo = (
    emendas_silver
    .select(F.col("ano").cast("int").alias("ano"))
    .dropDuplicates()
    .withColumn("sk_tempo", F.monotonically_increasing_id())
    .select(
        "sk_tempo",
        "ano"
    )
)

In [0]:
df_dim_tempo.display()

sk_tempo,ano
0,2023
1,2024
2,2025


In [0]:
(
    df_dim_tempo
    .write
    .mode("overwrite")
    .format("delta")
    .saveAsTable("mvp.gold.dim_tempo")
)

In [0]:
spark.sql("""
COMMENT ON TABLE mvp.gold.dim_tempo IS
'Dimensão tempo com granularidade anual, utilizada para análises temporais das emendas parlamentares.'
""")

DataFrame[]

In [0]:
spark.sql("""
COMMENT ON COLUMN mvp.gold.dim_tempo.sk_tempo IS
'Chave substituta (surrogate key) da dimensão tempo, gerada na camada Gold.'
""")

spark.sql("""
COMMENT ON COLUMN mvp.gold.dim_tempo.ano IS
'Ano de referência da emenda parlamentar. (Valores válidos: 2023, 2024 e 2025)'
""")

DataFrame[]

## Dim_Localidade (Onde)

### 🗺️ Dimensão: Dim_Localidade

| Coluna             | Descrição                                                        | Tipo de Dado | Observações |
|--------------------|------------------------------------------------------------------|--------------|-------------|
| sk_localidade      | Chave substituta (surrogate key) da dimensão de localidade.     | BIGINT       | Gerada na camada Gold |
| localidade_destino   | Nome original da localidade ou regionalização do gasto.         | STRING       | Informação conforme fonte |
| municipio_destino  | Nome do município de destino do recurso, quando aplicável.     | STRING       | Pode ser nula|
| uf_destino         | Sigla da Unidade da Federação de destino, quando aplicável.     | STRING       | Pode ser nula |



In [0]:
# Construção da Dim_Localidade
df_dim_localidade = (
    emendas_silver
    .select(
        F.col("localidade_destino"),
        F.col("municipio_destino"),
        F.col("uf_destino")
    )
    .dropDuplicates([
        "localidade_destino",
        "municipio_destino",
        "uf_destino"
    ])
    .withColumn("sk_localidade", F.monotonically_increasing_id())
    .select(
        "sk_localidade",
        "localidade_destino",
        "municipio_destino",
        "uf_destino"
    )
)

In [0]:
(
    df_dim_localidade
    .write
    .mode("overwrite")
    .format("delta")
    .saveAsTable("mvp.gold.dim_localidade")
)

In [0]:
spark.sql("""
COMMENT ON TABLE mvp.gold.dim_localidade IS
'Dimensão de localidade que descreve o destino geográfico dos recursos das emendas parlamentares, incluindo localidades nacionais, regionais, municipais ou estaduais.'
""")

DataFrame[]

In [0]:
spark.sql("""
COMMENT ON COLUMN mvp.gold.dim_localidade.sk_localidade IS
'Chave substituta (surrogate key) da dimensão de localidade, gerada na camada Gold.'
""")

spark.sql("""
COMMENT ON COLUMN mvp.gold.dim_localidade.localidade_destino IS
'Nome da localidade ou regionalização do gasto conforme informado na fonte de dados.'
""")

spark.sql("""
COMMENT ON COLUMN mvp.gold.dim_localidade.municipio_destino IS
'Nome do município de destino do recurso, quando aplicável. Pode ser nula para localidades estaduais, regionais ou nacionais.'
""")

spark.sql("""
COMMENT ON COLUMN mvp.gold.dim_localidade.uf_destino IS
'Sigla da Unidade da Federação de destino do recurso, quando aplicável. Pode ser nula para localidades nacionais ou regionais.'
""")

DataFrame[]

In [0]:
%sql
select *
from mvp.gold.dim_localidade
limit 5

sk_localidade,localidade_destino,municipio_destino,uf_destino
0,JAPARATINGA - AL,JAPARATINGA,AL
1,CRATO - CE,CRATO,CE
2,TAMBORIL - CE,TAMBORIL,CE
3,ITAPOROROCA - PB,ITAPOROROCA,PB
4,VOTORANTIM - SP,VOTORANTIM,SP


## Dim_Orçamento (Para quê)


### 💰 Dimensão: Dim_Orcamento

| Coluna        | Descrição                                                        | Tipo de Dado | Observações |
|---------------|------------------------------------------------------------------|--------------|-------------|
| sk_orcamento  | Chave substituta (surrogate key) da dimensão de orçamento.       | BIGINT       | Gerada na camada Gold |
| funcao        | Função orçamentária de alto nível que representa a área temática | STRING       | Coluna chave para análise |


In [0]:
# Construção da Dim_Orcamento
df_dim_orcamento = (
    emendas_silver
    .select(
        F.col("funcao").alias("funcao")
    )
    .filter(F.col("funcao").isNotNull())
    .dropDuplicates()
    .withColumn("sk_orcamento", F.monotonically_increasing_id())
    .select(
        "sk_orcamento",
        "funcao"
    )
)

In [0]:
(
    df_dim_orcamento
    .write
    .mode("overwrite")
    .format("delta")
    .saveAsTable("mvp.gold.dim_orcamento")
)

In [0]:
spark.sql("""
COMMENT ON TABLE mvp.gold.dim_orcamento IS
'Dimensão de orçamento que consolida as funções orçamentárias, representando a área temática de destinação dos recursos.'
""")


DataFrame[]

In [0]:
spark.sql("""
COMMENT ON COLUMN mvp.gold.dim_orcamento.sk_orcamento IS
'Chave substituta (surrogate key) da dimensão de orçamento, gerada na camada Gold.'
""")

spark.sql("""
COMMENT ON COLUMN mvp.gold.dim_orcamento.funcao IS
'Função orçamentária de alto nível que representa a área temática da destinação dos recursos (ex.: Saúde, Educação).'
""")

DataFrame[]

In [0]:
%sql
select *
from mvp.gold.dim_orcamento
limit 5

sk_orcamento,funcao
0,Múltiplo
1,Organização agrária
2,Desporto e lazer
3,Ciência e Tecnologia
4,Encargos especiais


# Criação da tabela fato

## Fato_Emenda

### 📊 Tabela Fato: Fato_Emenda

| Coluna            | Descrição                                               | Tipo de Dado        | Chave | Origem |
|-------------------|---------------------------------------------------------|---------------------|-------|--------|
| id_emenda         | Identificador único do registro da emenda.              | BIGINT              | PK    | Gerada |
| sk_parlamentar    | Chave estrangeira para a Dim_Parlamentar.               | BIGINT              | FK    | Dim_Parlamentar |
| sk_tempo          | Chave estrangeira para a Dim_Tempo.                     | BIGINT              | FK    | Dim_Tempo |
| sk_localidade     | Chave estrangeira para a Dim_Localidade.                | BIGINT              | FK    | Dim_Localidade |
| sk_orcamento      | Chave estrangeira para a Dim_Orcamento.                 | BIGINT              | FK    | Dim_Orcamento |
| valor_empenhado   | Valor total empenhado da emenda.                        | DECIMAL(18,2)       | Métrica | Silver |
| valor_liquidado   | Valor total liquidado da emenda.                        | DECIMAL(18,2)       | Métrica | Silver |
| valor_pago        | Valor total pago da emenda.                             | DECIMAL(18,2)       | Métrica | Silver |


In [0]:
df_dim_parlamentar = spark.read.table("mvp.gold.dim_parlamentar")
df_dim_tempo = spark.read.table("mvp.gold.dim_tempo")
df_dim_localidade = spark.read.table("mvp.gold.dim_localidade")
df_dim_orcamento = spark.read.table("mvp.gold.dim_orcamento")

In [0]:
df_fato_emenda = (
    emendas_silver
    # Parlamentar
    .join(
        df_dim_parlamentar,
        on="nome_parlamentar",
        how="inner"
    )
    # Tempo
    .join(
        df_dim_tempo,
        on="ano",
        how="left"
    )
    # Localidade
    .join(
        df_dim_localidade,
        on=["localidade_destino"],
        how="left"
    )
    # Orçamento
    .join(
        df_dim_orcamento,
        on="funcao",
        how="left"
    )
)

In [0]:
df_fato_emenda = (
    df_fato_emenda
    .select(
        F.monotonically_increasing_id().alias("id_emenda"),
        "sk_parlamentar",
        "sk_tempo",
        "sk_localidade",
        "sk_orcamento",
        F.col("valor_empenhado").cast("decimal(18,2)").alias("valor_empenhado"),
        F.col("valor_liquidado").cast("decimal(18,2)").alias("valor_liquidado"),
        F.col("valor_pago").cast("decimal(18,2)").alias("valor_pago")
    )
)

In [0]:
(
    df_fato_emenda
    .write
    .mode("overwrite")
    .format("delta")
    .saveAsTable("mvp.gold.fato_emenda")
)

In [0]:
spark.sql("""
COMMENT ON TABLE mvp.gold.fato_emenda IS
'Tabela fato de emendas parlamentares que registra as métricas financeiras e se conecta às dimensões de parlamentar, tempo, localidade e orçamento.'
""")

DataFrame[]

In [0]:
spark.sql("""
COMMENT ON COLUMN mvp.gold.fato_emenda.id_emenda IS
'Identificador único do registro da emenda, gerado na camada Gold.'
""")

spark.sql("""
COMMENT ON COLUMN mvp.gold.fato_emenda.sk_parlamentar IS
'Chave estrangeira para a Dim_Parlamentar.'
""")

spark.sql("""
COMMENT ON COLUMN mvp.gold.fato_emenda.sk_tempo IS
'Chave estrangeira para a Dim_Tempo.'
""")

spark.sql("""
COMMENT ON COLUMN mvp.gold.fato_emenda.sk_localidade IS
'Chave estrangeira para a Dim_Localidade.'
""")

spark.sql("""
COMMENT ON COLUMN mvp.gold.fato_emenda.sk_orcamento IS
'Chave estrangeira para a Dim_Orcamento.'
""")

spark.sql("""
COMMENT ON COLUMN mvp.gold.fato_emenda.valor_empenhado IS
'Valor financeiro empenhado da emenda parlamentar.'
""")

spark.sql("""
COMMENT ON COLUMN mvp.gold.fato_emenda.valor_liquidado IS
'Valor financeiro liquidado da emenda parlamentar.'
""")

spark.sql("""
COMMENT ON COLUMN mvp.gold.fato_emenda.valor_pago IS
'Valor financeiro efetivamente pago da emenda parlamentar.'
""")

DataFrame[]

In [0]:
# Verificar FKs nulas (problema de join)
spark.table("mvp.gold.fato_emenda") \
    .select(
        F.sum(F.col("sk_parlamentar").isNull().cast("int")).alias("parlamentar_null"),
        F.sum(F.col("sk_tempo").isNull().cast("int")).alias("tempo_null"),
        F.sum(F.col("sk_localidade").isNull().cast("int")).alias("localidade_null"),
        F.sum(F.col("sk_orcamento").isNull().cast("int")).alias("orcamento_null")
    ) \
    .display()

parlamentar_null,tempo_null,localidade_null,orcamento_null
0,0,0,0


In [0]:
%sql
select *
from mvp.gold.fato_emenda
limit 5

id_emenda,sk_parlamentar,sk_tempo,sk_localidade,sk_orcamento,valor_empenhado,valor_liquidado,valor_pago
0,330,1,968,22,614604.00,0.00,0.00
1,485,1,743,21,12113873.00,4854777.00,4854777.00
2,243,1,627,21,3836834.98,1720960.07,527236.07
3,317,1,743,21,5145725.00,0.00,0.00
4,215,0,181,17,249923.36,231968.87,228774.79
